# Machine Learning

This part will train additional networks to predict C-beta positions as well.

We begin with some initial classes and functions to load data

In [ ]:
import random
import numpy as np
import torch
import sys
import pandas as pd

class Dataset(torch.utils.data.Dataset):
# optional parameters allow target data to be shifted and scaled
    def __init__(self, target, data):
        self.label = target.astype(np.float32)
# assumes that data is prepared in correct shape beforehand
        self.input = data
    def __len__(self):
        return self.label.shape[0]
    def __getitem__(self, index):
        return self.input[index].astype(np.float32), self.label[index]
    
def randomsplitdata(target_input_fn,training_fraction,maxrows=-1,select="CACNO",aa='all'):
    f=open(target_input_fn)
    lsplit=f.readline().split()
    f.close()

    if (maxrows>0):
        df=pd.read_csv(target_input_fn,sep=' ',header=None,nrows=maxrows)
    else:
        df=pd.read_csv(target_input_fn,sep=' ',header=None)
        
    if (aa == 'all'):
        dfaa=df
    else:
        aalist=aa.split(':')
        dfaa=df.loc[df[0].isin(aalist)]
       
    if (select == "CACNO"): 
        dft=dfaa[[*range(1,13)]]                                                 #ca, c, n, o
    elif (select == "CANCO"):
        dft=dfaa[[*range(1,4)]+[*range(7,10)]+[*range(4,7)]+[*range(10,13)]]     #ca, n, c, o
    elif (select == "CAN"):
        dft=dfaa[[*range(1,4)]+[*range(7,10)]]                                   #ca, n
    elif (select == "CAC"):
        dft=dfaa[[*range(1,7)]]                                                  #ca, c
    elif (select == "CACN"):
        dft=dfaa[[*range(1,10)]]                                                 #ca, c, n
    elif (select == "CA"):
        dft=dfaa[[*range(1,4)]]
    elif (select == "CACNOCB"):
        dft=dfaa[[*range(1,16)]]
    elif (select == "CANCOCB"):
        dft=dfaa[[*range(1,4)]+[*range(7,10)]+[*range(4,7)]+[*range(10,16)]]     #ca, n, c, o
    else:
        print('unknown selection')
        sys.exit(1)
      
    targetdata=dft.to_numpy()
    
    print(targetdata.shape)

    dfi=dfaa[[*range(len(lsplit)-30,len(lsplit))]]
    inputdata=np.reshape(dfi.to_numpy(),(len(dfi),1,-1))
      
    xlist=[*range(len(lsplit)-30,len(lsplit),3)]
    ylist=[*range(len(lsplit)-30+1,len(lsplit),3)]
    zlist=[*range(len(lsplit)-30+2,len(lsplit),3)]
    
    dfixyz=dfi[xlist+ylist+zlist]
    inpxyzdata=np.reshape(dfixyz.to_numpy(),(len(dfi),3,-1)) 
    
    flag=np.zeros(len(targetdata),dtype=int)
    while np.average(flag)<training_fraction:
        flag[random.randint(0,len(targetdata)-1)]=1
    
    target_training=targetdata[np.nonzero(flag)].copy()
    target_validation=targetdata[np.nonzero(1-flag)].copy()
    input_training=inputdata[np.nonzero(flag)].copy()
    input_validation=inputdata[np.nonzero(1-flag)].copy()
    inputxyz_training=inpxyzdata[np.nonzero(flag)].copy()
    inputxyz_validation=inpxyzdata[np.nonzero(1-flag)].copy()
        
    return target_training,input_training,inputxyz_training, \
           target_validation,input_validation,inputxyz_validation

def get_loaders(target_input_fn,training_fraction,batch_size=128,maxrows=-1,aa='all',select="CACNO"):
    [ttarget,tinput,tinpxyz,vtarget,vinput,vinpxyz]=\
        randomsplitdata(target_input_fn,training_fraction,maxrows=maxrows,aa=aa,select=select) 
    
    train_set=Dataset(ttarget,tinput)
    validation_set=Dataset(vtarget,vinput)
    trainxyz_set=Dataset(ttarget,tinpxyz)
    valixyz_set=Dataset(vtarget,vinpxyz)
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=1)
    
    trainxyz_loader = torch.utils.data.DataLoader(trainxyz_set, batch_size=batch_size, shuffle=True)
    valixyz_loader = torch.utils.data.DataLoader(valixyz_set, batch_size=1)
    
    return train_loader,validation_loader,trainxyz_loader,valixyz_loader        

We set up training functions again.

In [ ]:
import numpy as np

def train(m,loss_fn,opt,loader,xscale=1.0,klfactor=0.0,klindex=0):
    klloss=nn.KLDivLoss(reduction='batchmean')
    
    loss_sum = 0.0
    for input, label in loader:
        opt.zero_grad()
        
        output = m(input)               # this is where the model is evaluated
        
        loss =  loss_fn(output, label)  # model loss for x       
        
        if (xscale>1.0000001 or xscale<0.9999999):
            nval=output.size(dim=1)
            for n in (range(0,nval,3)):
                loss += loss_fn(output[:,n], label[:,n])*(xscale-1.0)
        
        loss_sum += loss.item()         # accumulate MSE loss
            
        if (klfactor>0):
            loss=loss+klfactor*\
                klloss(output[:,klindex],label[:,klindex])
            
        loss.backward()                 # this calculates the back-propagated loss
        opt.step()                      # this carries out the gradient descent
    
    return loss_sum / len(loader)       # Note: KL loss is not included in reported loss

def validate(m,loss_fn,loader,xscale=1.0):
    loss_sum = 0.0
    for input, label in loader:
        with torch.no_grad():
            output = m(input)

        loss = loss_fn(output, label)

        if (xscale>1.0000001 or xscale<0.9999999):
            nval=output.size(dim=1)
            for n in (range(0,nval,3)):
                loss += loss_fn(output[:,n], label[:,n])*(xscale-1.0)
        
        loss_sum += loss.item()
    return loss_sum / len(loader)

def do_training(m,opt,tloader,vloader,epochs,output,xscale=1.0,klfactor=0.0,klindex=0):
    # use MSE loss fucntion
    loss_fn = nn.MSELoss()
    
    tloss=np.zeros(epochs)
    vloss=np.zeros(epochs)

    for i in range(epochs):
        tloss[i] = train(m,loss_fn,opt,tloader,xscale,klfactor,klindex)
        vloss[i] = validate(m,loss_fn,vloader,xscale)
        if (output):
            print (i, tloss[i], vloss[i])
            
    return tloss,vloss


We also define some plotting functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

def plot_progress(epochs,tloss,vloss):
    plt.rcParams["figure.figsize"]=(6,4)
    epoch_index=np.arange(epochs)
    plt.plot(epoch_index,np.log(tloss),color='r',label='training')
    plt.plot(epoch_index,np.log(vloss),color='b',label='validation')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.show()

def plot_validation(loader,m,inx=0):    
    targetx=[]
    targety=[]
    targetz=[]
    
    predictionx=[]
    predictiony=[]
    predictionz=[]

    for input, label in loader:        
        with torch.no_grad():
            output = m(input)
                 
        targetx+=[label[0,inx*3+0].item()]
        targety+=[label[0,inx*3+1].item()]
        targetz+=[label[0,inx*3+2].item()]
        
        predictionx+=[output[0,inx*3+0].item()]
        predictiony+=[output[0,inx*3+1].item()]
        predictionz+=[output[0,inx*3+2].item()]
    
    plt.rcParams["figure.figsize"]=(12,3)

    fig,ax = plt.subplots(1,3)
    
    minval=np.min(targetx)
    maxval=np.max(targetx)
    lin=np.linspace(minval-0.1*(maxval-minval),maxval+0.1*(maxval-minval),num=100)
    
    ax[0].plot(lin,lin,'k',linewidth=2)
    ax[0].plot(targetx,predictionx,'ro',markersize=2)
    ax[0].set(xlabel='target x [nm]', ylabel="prediction [nm]")
    
    minval=np.min(targety)
    maxval=np.max(targety)
    lin=np.linspace(minval-0.1*(maxval-minval),maxval+0.1*(maxval-minval),num=100)

    ax[1].plot(lin,lin,'k',linewidth=2)
    ax[1].plot(targety,predictiony,'ro',markersize=2)
    ax[1].set(xlabel='target y [nm]', ylabel="")

    minval=np.min(targetz)
    maxval=np.max(targetz)
    lin=np.linspace(minval-0.1*(maxval-minval),maxval+0.1*(maxval-minval),num=100)

    ax[2].plot(lin,lin,'k',linewidth=2)
    ax[2].plot(targetz,predictionz,'ro',markersize=2)    
    ax[2].set(xlabel='target z [nm]', ylabel="")
    
    plt.show()
    
def linear_regression_output(x,y,txt):
    x=np.reshape(np.array(x),(-1,1))
    y=np.reshape(np.array(y),(-1,1))
    linmodel=LinearRegression().fit(x,y)
    r2=linmodel.score(x,y)
    mval=linmodel.coef_[0]
    nval=linmodel.intercept_
    print(f'{txt}: r2 {r2} slope {mval[0]} offset {nval[0]}')
    
def linear_regression(loader,m,inx=0):    
    targetx=[]
    targety=[]
    targetz=[]
    
    predictionx=[]
    predictiony=[]
    predictionz=[]

    for input, label in loader:        
        with torch.no_grad():
            output = m(input)
                 
        targetx+=[label[0,inx*3+0].item()]
        targety+=[label[0,inx*3+1].item()]
        targetz+=[label[0,inx*3+2].item()]
        
        predictionx+=[output[0,inx*3+0].item()]
        predictiony+=[output[0,inx*3+1].item()]
        predictionz+=[output[0,inx*3+2].item()]
    
    linear_regression_output(targetx,predictionx,"x")
    linear_regression_output(targety,predictiony,"y")
    linear_regression_output(targetz,predictionz,"z")

Next we define the neural network models

Model1D3_ca_n_co_cb extends Model1D3_ca_n_co_cb to first predict CA,N,C/O using previously trained weights and then to predict CB

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F          
    
class Model1D3_ca_n_co_cb(nn.Module):
    def __init__(self):
        super(Model1D3_ca_n_co_cb, self).__init__()
        # define layers to be used
        self.conv_1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv_3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv_4 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv_f = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # dimensional flattening
        self.flatten = nn.Flatten(start_dim=1) 
        # fully connected layers
        self.fc_1 = nn.Linear(640,128)
        self.fc_2 = nn.Linear(128,32)
        self.fc_f = nn.Linear(32, 3)

        self.cn_1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.cn_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cn_3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cn_4 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cn_f = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # fully connected layers
        self.fcn_1 = nn.Linear(704,128)
        self.fcn_2 = nn.Linear(128,32)
        self.fcn_f = nn.Linear(32, 3)
        
        self.cco_1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.cco_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cco_3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cco_4 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.cco_f = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # fully connected layers
        self.fcco_1 = nn.Linear(768,128)
        self.fcco_2 = nn.Linear(128,32)
        self.fcco_f = nn.Linear(32, 6)

        self.ccb_1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.ccb_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.ccb_3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.ccb_4 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.ccb_f = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # fully connected layers
        self.fccb_1 = nn.Linear(896,128)
        self.fccb_2 = nn.Linear(128,32)
        self.fccb_f = nn.Linear(32, 3)

    def forward(self, x):
        # back-propagation is done automatically
        xinp=x
        x = self.conv_1(xinp)
        x = F.relu(self.conv_2(x))
        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = F.relu(self.conv_f(x))
        x = self.flatten(x)
        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x)) 
        ca = self.fc_f(x)       
       
        car=ca.reshape(-1,3,1)
        x = torch.cat((car,xinp),2)
        x = self.cn_1(x)
        x = F.relu(self.cn_2(x))
        x = F.relu(self.cn_3(x))
        x = F.relu(self.cn_4(x))
        x = F.relu(self.cn_f(x))
        x = self.flatten(x)
        x = F.relu(self.fcn_1(x))
        x = F.relu(self.fcn_2(x)) 
        n = self.fcn_f(x)       
        
        nr=n.reshape(-1,3,1)
        x = torch.cat((car,nr,xinp),2)
        x = self.cco_1(x)
        x = F.relu(self.cco_2(x))
        x = F.relu(self.cco_3(x))
        x = F.relu(self.cco_4(x))
        x = F.relu(self.cco_f(x))
        x = self.flatten(x)
        #print(x.size())
        x = F.relu(self.fcco_1(x))
        x = F.relu(self.fcco_2(x)) 
        co = self.fcco_f(x)       
        c = co[:,0:3]
        ox = co[:,3:6]
        
        cr=c.reshape(-1,3,1)
        oxr=ox.reshape(-1,3,1)
        x = torch.cat((car,nr,cr,oxr,xinp),2)
        x = self.ccb_1(x)
        x = F.relu(self.ccb_2(x))
        x = F.relu(self.ccb_3(x))
        x = F.relu(self.ccb_4(x))
        x = F.relu(self.ccb_f(x))
        x = self.flatten(x)
        #print(x.size())
        x = F.relu(self.fccb_1(x))
        x = F.relu(self.fccb_2(x)) 
        x = self.fccb_f(x)       
               
        x = torch.cat((ca,n,c,ox,x),1)
        return x
    
    def initialize_weights(self, m):
        # initialization of weights, setting them to zero is not good
        if hasattr(m, 'weight') and m.weight.dim() > 1:
            nn.init.xavier_uniform_(m.weight.data)


Now we are training the models using the pretrained weights for specific amino acids.

In [ ]:
aa='ALA'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=1.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='PRO'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=1.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='VAL'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='LEU'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='PHE'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='ILE'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=10
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='TYR'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='TRP'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='ASP'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='ASN'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='GLU'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='GLN'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=3.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='TYR'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='CYS'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=30
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='HIS'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='LYS'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
#m.load_state_dict(torch.load(f"{aa}_1d3_ca_n_co_cb_predict_aa.dict"))
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)


epochs=20
#epochs=5
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='MET'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='SER'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")


In [ ]:
aa='THR'

[tloader,vloader,txyzloader,vxyzloader]=\
    get_loaders('../generatingfeatures/longer_local_i_aa_capm2opmnpmcpm.dat',0.8,\
               batch_size=512,aa=aa,select="CANCOCB") 

m = Model1D3_ca_n_co_cb()
m.apply(m.initialize_weights)
m.zero_grad()

# load partial weights from previous model
# may use other models previously trained 
m.load_state_dict(torch.load(f'{aa}_1d3_ca_n_co_predict_aa.dict'),strict=False)

# disable training of all weights
for param in m.parameters():
    param.requires_grad = False

# reenable training of only the second part of the network that predicts Cbetas 
# this means that the CA/N/CO prediction is not changed during training
for param in m.ccb_1.parameters():
    param.requires_grad = True
for param in m.ccb_2.parameters():
    param.requires_grad = True
for param in m.ccb_3.parameters():
    param.requires_grad = True
for param in m.ccb_4.parameters():
    param.requires_grad = True
for param in m.ccb_f.parameters():
    param.requires_grad = True   
for param in m.fccb_1.parameters():
    param.requires_grad = True
for param in m.fccb_2.parameters():
    param.requires_grad = True
for param in m.fccb_f.parameters():
    param.requires_grad = True

# optimizer with extra options to only train certain weights
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.0005, weight_decay=0.00001)

epochs=20
showoutput=True

[tloss,vloss]=do_training(m,opt,txyzloader,vxyzloader,epochs,showoutput,xscale=5.0,klfactor=2.0,klindex=12)

plot_progress(epochs,tloss,vloss)

print('CA')
plot_validation(vxyzloader,m,0)
linear_regression(vxyzloader,m,0)

print('N')
plot_validation(vxyzloader,m,1)
linear_regression(vxyzloader,m,1)

print('C')
plot_validation(vxyzloader,m,2)
linear_regression(vxyzloader,m,2)

print('O')
plot_validation(vxyzloader,m,3)
linear_regression(vxyzloader,m,3)

print('CB')
plot_validation(vxyzloader,m,4)
linear_regression(vxyzloader,m,4)


torch.save(m.state_dict(),f"{aa}_1d3_ca_n_co_cb_predict_aa.dict")
